In [ ]:

groupby_columns = fu.scene_groupby_columns
error_types_df = csv_stats_df.copy()
for groupby_tuple, responder_categories_df in error_types_df.groupby(groupby_columns):
    break

In [ ]:

{column_name: column_value for column_name, column_value in zip(groupby_columns, groupby_tuple)}


# Verification

In [ ]:

mask_series = ~scene_df.patient_id.isnull()
df = scene_df[mask_series]
if df.shape[0]: display(df.sample(min(df.shape[0], 5)).dropna(axis='columns', how='all').T)

In [ ]:

# Get them set up as if we were going to run triage accuracy and time to hemorrhage control stats on them
# I know they are small samples, I primarily want you to let us know what information is missing for you to look at those things.
# I am also interested in variability around each patient. Are you able to segregate the files by patient and tell me something about
# how different participants acted (assess, treat, tag) in response to each patient?
# I also want to extract from each csv a list of the actions the responder did (engage, assess, treat, and tag actions, not movement or others)
# so that I can see a list of what they did in the scenario (this will eventually form the basis of delegation decisions).
# Basically I would like you to play with this small sample in anticipation of receiving a similar, larger dataset from these new scenarios.
# Let me know if you have questions but also please be patient as this is low priority compared to evaluation preparation this week.
import os
import glob
import os.path as osp

data_frames_dict = nu.load_data_frames(verbose=False, metrics_evaluation_open_world_scene_stats_df='', metrics_evaluation_open_world_json_stats_df='')
json_stats_df = data_frames_dict['metrics_evaluation_open_world_json_stats_df']
logs_folder = '../data/logs'
pattern = osp.join(logs_folder, 'Disaster Day 3.6.2024 ITM Files 405*')
csv_count = 0
for logs_path in glob.glob(pattern):
    csv_count += len([fn for sub_directory, directories_list, files_list in os.walk(logs_path) for fn in files_list if fn.lower().endswith('.csv')])
uuids_count = json_stats_df.session_uuid.unique().shape[0]
elevens_mask = (scene_stats_df.patient_count >= 11) & (scene_stats_df.is_scene_aborted == False)
scene_type_dict = scene_stats_df[elevens_mask].groupby('scene_type').size().to_dict()
orientation_count = scene_type_dict['Orientation'] if 'Orientation' in scene_type_dict else 0
triage_count = scene_type_dict['Triage'] if 'Triage' in scene_type_dict else 0
aborted_count = scene_stats_df[scene_stats_df.is_scene_aborted].shape[0]
aborted_suffix = 's' if (aborted_count != 1) else ''
one_triage_dict = json_stats_df.groupby('is_a_one_triage_file').size().to_dict()
print(f'''Okay, I have ingested Disaster day 3.6.2024 ITM files 405F.zip and Disaster Day 3.6.2024 ITM Files 405E.zip and have verified that:
•	There are {csv_count} CSV files in the zip file.
•	There are {uuids_count} unique session UUIDs in there.
•	There are {orientation_count} 11-patient-or-above Orientation scenes and {triage_count} Triage scenes in there.
•	The time difference between when the scene starts and the last engagement for {aborted_count} scene{aborted_suffix} is longer than 16 minutes.
•	There are {one_triage_dict.get(True, 0)} files that have one and only one triage scene in them. (The other {one_triage_dict.get(False, 0)}''')
print(''' are not one-triage-scene files).
If any of these is unexpected, please explain why.
''')
mask_series = (scene_stats_df.scene_type == 'Orientation') & elevens_mask
uuids_list = sorted(scene_stats_df[mask_series].session_uuid.unique())
mask_series = json_stats_df.session_uuid.isin(uuids_list)
if mask_series.any():
    print(
        f'P.S. The files with orientation scenes in them are:'
    )
    for file_name in sorted(json_stats_df[mask_series].session_file_name): print(file_name)

In [ ]:

from IPython.display import HTML

if 'is_in_registry' in json_stats_df:
    registry_uuids = sorted(json_stats_df[json_stats_df.is_in_registry].session_uuid.unique())
    print(f'Of the {len(registry_uuids)} files in the registry, ', end='')
    mask_series = scene_stats_df.session_uuid.isin(registry_uuids) & (scene_stats_df.scene_type == 'Triage')
    print(f'of the {scene_stats_df[mask_series].shape[0]} triage scenes in those files, this is the counts of the values of responder type:')
    display(HTML(json_stats_df.responder_type.value_counts().to_frame().to_html()))

In [ ]:

display(HTML(scene_stats_df.groupby(['patient_count', 'scene_type']).size().to_frame().rename(columns={0: 'record_count'}).reset_index(drop=False).to_html()))

In [ ]:

mask_series = (scene_stats_df.scene_type == 'Orientation')
df = scene_stats_df[mask_series].dropna(axis='columns', how='all')
display(df.sample(10).T)
if 'session_uuid' in df.columns:
    session_uuids_list = sorted(df.session_uuid.unique())
    mask_series = json_stats_df.session_uuid.isin(session_uuids_list)
    df = json_stats_df[mask_series].dropna(axis='columns', how='all')
    display(df.sample(9).T)


----

In [ ]:

print()
is_scalar_lambda = lambda value: isinstance(value, (int, float, str, bool, np.number))
for (session_uuid, scene_id), idx_df in distance_delta_df.groupby(fu.scene_groupby_columns):
    
    # Get the whole scene history
    mask_series = True
    for cn in fu.scene_groupby_columns: mask_series &= (csv_stats_df[cn] == eval(cn))
    scene_df = csv_stats_df[mask_series]
    
    if is_scalar_lambda(fu.get_scene_start(scene_df)): print(
        "    row_dict['scene_start'] = fu.get_scene_start(scene_df)"
    )
    if is_scalar_lambda(fu.get_last_engagement(scene_df)): print(
        "    row_dict['last_engagement'] = fu.get_last_engagement(scene_df)"
    )
    if is_scalar_lambda(fu.get_player_location(scene_df, action_tick=0)): print(
        "    row_dict['player_location'] = fu.get_player_location(scene_df, action_tick)"
    )
    if is_scalar_lambda(fu.get_scene_type(scene_df)): print(
        "    row_dict['scene_type'] = fu.get_scene_type(scene_df)"
    )
    if is_scalar_lambda(fu.get_scene_end(scene_df)): print(
        "    row_dict['scene_end'] = fu.get_scene_end(scene_df)"
    )
    if is_scalar_lambda(fu.get_patient_count(scene_df)): print(
        "    row_dict['patient_count'] = fu.get_patient_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_injury_treatments_count(scene_df)): print(
        "    row_dict['injury_treatments_count'] = fu.get_injury_treatments_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_injury_not_treated_count(scene_df)): print(
        "    row_dict['injury_not_treated_count'] = fu.get_injury_not_treated_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_injury_correctly_treated_count(scene_df)): print(
        "    row_dict['injury_correctly_treated_count'] = fu.get_injury_correctly_treated_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_injury_wrongly_treated_count(scene_df)): print(
        "    row_dict['injury_wrongly_treated_count'] = fu.get_injury_wrongly_treated_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_pulse_taken_count(scene_df)): print(
        "    row_dict['pulse_taken_count'] = fu.get_pulse_taken_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_teleport_count(scene_df)): print(
        "    row_dict['teleport_count'] = fu.get_teleport_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_voice_capture_count(scene_df)): print(
        "    row_dict['voice_capture_count'] = fu.get_voice_capture_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_walk_command_count(scene_df)): print(
        "    row_dict['walk_command_count'] = fu.get_walk_command_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_wave_command_count(scene_df)): print(
        "    row_dict['wave_command_count'] = fu.get_wave_command_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_first_engagement(scene_df)): print(
        "    row_dict['first_engagement'] = fu.get_first_engagement(scene_df)"
    )
    if is_scalar_lambda(fu.get_first_treatment(scene_df)): print(
        "    row_dict['first_treatment'] = fu.get_first_treatment(scene_df)"
    )
    if is_scalar_lambda(fu.get_ideal_engagement_order(scene_df, tuples_list=None)): print(
        "    row_dict['ideal_engagement_order'] = fu.get_ideal_engagement_order(scene_df, tuples_list=None)"
    )
    if is_scalar_lambda(fu.get_is_scene_aborted(scene_df)): print(
        "    row_dict['is_scene_aborted'] = fu.get_is_scene_aborted(scene_df)"
    )
    if is_scalar_lambda(fu.get_triage_time(scene_df)): print(
        "    row_dict['triage_time'] = fu.get_triage_time(scene_df)"
    )
    if is_scalar_lambda(fu.get_dead_patients(scene_df)): print(
        "    row_dict['dead_patients'] = fu.get_dead_patients(scene_df)"
    )
    if is_scalar_lambda(fu.get_still_patients(scene_df)): print(
        "    row_dict['still_patients'] = fu.get_still_patients(scene_df)"
    )
    if is_scalar_lambda(fu.get_total_actions_count(scene_df)): print(
        "    row_dict['total_actions'] = fu.get_total_actions_count(scene_df)"
    )
    if is_scalar_lambda(fu.get_actual_and_ideal_patient_sort_sequences(scene_df)): print(
        "    row_dict['actual_and_ideal_sequences'] = fu.get_actual_and_ideal_patient_sort_sequences(scene_df)"
    )
    if is_scalar_lambda(fu.get_measure_of_right_ordering(scene_df)): print(
        "    row_dict['measure_of_right_ordering'] = fu.get_measure_of_right_ordering(scene_df)"
    )
    if is_scalar_lambda(fu.get_percent_hemorrhage_controlled(scene_df)): print(
        "    row_dict['percent_hemorrhage_controlled'] = fu.get_percent_hemorrhage_controlled(scene_df)"
    )
    if is_scalar_lambda(fu.get_time_to_last_hemorrhage_controlled(scene_df)): print(
        "    row_dict['time_to_last_hemorrhage_controlled'] = fu.get_time_to_last_hemorrhage_controlled(scene_df)"
    )
    if is_scalar_lambda(fu.get_time_to_hemorrhage_control_per_patient(scene_df)): print(
        "    row_dict['time_to_hemorrhage_control_per_patient'] = fu.get_time_to_hemorrhage_control_per_patient(scene_df)"
    )
    if is_scalar_lambda(fu.get_triage_priority_data_frame(scene_df)): print(
        "    row_dict['triage_priority_data_frame'] = fu.get_triage_priority_data_frame(scene_df)"
    )
    if is_scalar_lambda(fu.get_actual_engagement_order(scene_df)): print(
        "    row_dict['actual_engagement_order'] = fu.get_actual_engagement_order(scene_df)"
    )
    if is_scalar_lambda(fu.get_distracted_engagement_order(scene_df, tuples_list=None)): print(
        "    row_dict['distracted_engagement_order'] = fu.get_distracted_engagement_order(scene_df, tuples_list=None)"
    )
    if is_scalar_lambda(fu.get_stills_value(scene_df)): print(
        "    row_dict['stills_value'] = fu.get_stills_value(scene_df)"
    )
    if is_scalar_lambda(fu.get_walkers_value(scene_df)): print(
        "    row_dict['walkers_value'] = fu.get_walkers_value(scene_df)"
    )
    if is_scalar_lambda(fu.get_wave_value(scene_df)): print(
        "    row_dict['wave_value'] = fu.get_wave_value(scene_df)"
    )
    if is_scalar_lambda(fu.get_walk_value(scene_df)): print(
        "    row_dict['walk_value'] = fu.get_walk_value(scene_df)"
    )
    break


Please use the data attached for exploratory analyses in our open world scenarios. The document attached defines each of the variables (by column) and each row represents one human.

You should have TWO simulation data files for each participant (row) and they should be matched up by the UUIDs in columns 3 and 4 (they should be two different environments in case you have those in separate folders).

I would like you to look for any patterns in these data, especially related to the variables “ST_KDMA_Text” “ST_KDMA_Sim” “AD_KDMA_Text” “AD_KDMA_Sim”
Let me know if you have any questions.


In [ ]:

# Add file start time
if ('in_sim1' not in json_stats_df.columns) or ('in_sim2' not in json_stats_df.columns):
    json_stats_df['in_sim1'] = False
    json_stats_df['in_sim2'] = False
    
    from pandas import read_excel
    file_path = '../data/xlsx/Metrics_Eval_Participant_data_for_BBAI.xlsx'
    participant_data_df = read_excel(file_path)
    
    mask_series = ~participant_data_df.Sim1.isnull() & (participant_data_df.Sim1 != '')
    sim1_session_uuids_list = [u[:-1] if u.endswith('_') else u for u in sorted(participant_data_df[mask_series].Sim1)]
    mask_series = ~participant_data_df.Sim2.isnull() & (participant_data_df.Sim2 != '')
    sim2_session_uuids_list = [u[:-1] if u.endswith('_') else u for u in sorted(participant_data_df[mask_series].Sim2)]
    assert set(sim1_session_uuids_list).intersection(set(sim2_session_uuids_list)) == set(), "The participant ID groupings are not distinct"
    
    mask_series = json_stats_df.session_uuid.isin(sim1_session_uuids_list)
    json_stats_df.loc[mask_series, 'in_sim1'] = True
    mask_series = json_stats_df.session_uuid.isin(sim2_session_uuids_list)
    json_stats_df.loc[mask_series, 'in_sim2'] = True
    
    nu.store_objects(metrics_evaluation_open_world_json_stats_df=json_stats_df, verbose=True)
    nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=json_stats_df, verbose=True)
display(json_stats_df.groupby('in_sim1', dropna=False).size().to_frame().rename(columns={0: 'record_count'}))
display(json_stats_df.groupby('in_sim2', dropna=False).size().to_frame().rename(columns={0: 'record_count'}))

In [ ]:

raise


# Verify

In [ ]:

# See if you can merge on Sim1's session_uuid
sim1_df = json_stats_df.merge(participant_data_df, left_on='session_uuid', right_on='Sim1', how='inner')
print(sim1_df.shape)
mask_series = (sim1_df.participant_id_x != sim1_df.participant_id_y)
assert mask_series.sum() == 0, f"{mask_series.sum()} files have mismatched Participant IDs in Sim1"

In [ ]:

# See if you can merge on Sim2's session_uuid
sim2_df = json_stats_df.merge(participant_data_df, left_on='session_uuid', right_on='Sim2', how='inner')
print(sim2_df.shape)
mask_series = (sim2_df.participant_id_x != sim2_df.participant_id_y)
assert mask_series.sum() == 0, f"{mask_series.sum()} files have mismatched Participant IDs in Sim2"


# Visualize

In [ ]:

print(json_stats_df.participant_id.nunique()) # 22

In [ ]:

# Show numeric columns
column_descriptions_df = nu.get_column_descriptions(participant_data_df)
mask_series = column_descriptions_df.dtype.isin(['float64', 'int64'])
columns_set = set(column_descriptions_df[mask_series].column_name).difference(set(['ParticipantID']))
list(columns_set)

In [ ]:

# Walk through the logs, getting only file names and session_uuids
from pandas import DataFrame, read_csv, concat
import os
from os import path as osp

important_columns_list = ['file_name', 'session_uuid']
file_names_and_session_uuids_df = DataFrame([], columns=important_columns_list)

# Iterate over the subdirectories, directories, and files in the logs folder
logs_folder = '../data/logs'
for sub_directory, directories_list, files_list in os.walk(logs_folder):
    
    # Create a data frame to store the data for the current subdirectory
    sub_directory_df = DataFrame([], columns=important_columns_list)
    
    # Iterate over the files in the current subdirectory
    for file_name in files_list:
        
        # If the file is a CSV file, merge it into the subdirectory data frame
        if file_name.endswith('.csv'):
            # sub_directory_df = fu.process_files(sub_directory_df, sub_directory, file_name, verbose=verbose)
            
            # Construct the full path to the file
            file_path = osp.join(sub_directory, file_name)
            
            # Attempt to read CSV file using pandas
            try: file_df = read_csv(file_path, header=None, index_col=False)
            
            # If unsuccessful, try using a reader
            except:
                rows_list = []
                with open(file_path, 'r') as f:
                    import csv
                    reader = csv.reader(f, delimiter=',', quotechar='"')
                    for values_list in reader:
                        if (values_list[-1] == ''): values_list.pop(-1)
                        rows_list.append({i: v for i, v in enumerate(values_list)})
                file_df = DataFrame(rows_list)
            
            # Ignore small files and return the subdirectory data frame unharmed
            if (file_df.shape[1] >= 16):
                
                # Add file name  to the data frame
                file_dir_suffix = osp.abspath(sub_directory).replace(osp.abspath(logs_folder) + os.sep, '')
                file_df['file_name'] = '/'.join(file_dir_suffix.split(os.sep)) + '/' + file_name
                
                # Name the global columns
                columns_list = ['action_type', 'action_tick', 'event_time', 'session_uuid']
                file_df.columns = columns_list + file_df.columns.tolist()[len(columns_list):]

                # Remove all but the file name and session columns
                file_df = file_df[important_columns_list].drop_duplicates()
                
                # Append the data frame for the current file to the data frame for the current subdirectory
                sub_directory_df = concat([sub_directory_df, file_df], axis='index')
    
    # Append the data frame for the current subdirectory to the main data frame
    file_names_and_session_uuids_df = concat([file_names_and_session_uuids_df, sub_directory_df], axis='index')
    
file_names_and_session_uuids_df = file_names_and_session_uuids_df.reset_index(drop=True)
mask_series = file_names_and_session_uuids_df.session_uuid.isin(sim_session_uuids_list)
df = file_names_and_session_uuids_df[mask_series]
print(df.shape) # (43, 3)
display(df.sample(5).T)

In [ ]:

import re

search_regex = re.compile('Urban', re.IGNORECASE)
columns_list = nu.get_regexed_columns(json_stats_df, search_regex=search_regex)
df = nu.get_regexed_dataframe(json_stats_df, columns_list, search_regex=search_regex)
df.sample(4)[columns_list].dropna(axis='columns', how='all').T

In [ ]:

columns_list = [
    'encounter_layout', 'configData_narrative_narrativeDescription', 'configData_scene', 'configData_scenarioData_name', 'configData_scenarioData_description'
]
for cn in columns_list: display(json_stats_df.groupby(cn, dropna=False).size().to_frame().rename(columns={0: 'record_count'}))

In [ ]:

def show_subgraph(sub_graph, suptitle='Within-function Function Calls', nodes_list_list=None, node_color='b', verbose=False):
    
    # Vertically separate the labels for easier readability
    layout_items = nx.spring_layout(sub_graph).items()
    left_lim, right_lim = -1500, 1500
    bottom_lim, top_lim = left_lim * nu.twitter_aspect_ratio, right_lim * nu.twitter_aspect_ratio
    rows_list = [{'node_name': node_name, 'layout_x': pos_array[0], 'layout_y': pos_array[1]} for node_name, pos_array in layout_items]
    df = DataFrame(rows_list).sort_values('layout_x')
    df['x_tick'] = [int(round(el)) for el in pd.cut(np.array([left_lim, right_lim]), len(sub_graph.nodes)+1, retbins=True)[1]][1:-1]
    df = df.sort_values('layout_y')
    df['y_tick'] = [int(round(el)) for el in pd.cut(np.array([bottom_lim, top_lim]), len(sub_graph.nodes)+1, retbins=True)[1]][1:-1]
    
    # Create the layout dictionary
    layout_dict = {}
    for row_index, row_series in df.iterrows():
        node_name = row_series.node_name
        layout_x = row_series.x_tick
        layout_y = row_series.y_tick
        layout_dict[node_name] = np.array([float(layout_x), float(layout_y)])
    
    # Draw the graph using the layout
    fig = plt.figure(figsize=(18, 7), facecolor='white')
    ax = fig.add_subplot(111)
    plt.axis('off')
    plt.xticks([], [])
    plt.yticks([], [])
    fig.suptitle(suptitle, fontsize=24)
    
    # Make the nodes the node_color
    if nodes_list_list is None:
        node_collection = nx.draw_networkx_nodes(
            G=sub_graph, pos=layout_dict, alpha=0.33, node_color=node_color.reshape(1, -1), node_size=150
        )
        edge_collection = nx.draw_networkx_edges(
            G=sub_graph, pos=layout_dict, alpha=0.25, width=[edge_tuple[2]['weight'] for edge_tuple in sub_graph.edges(data=True)]
        )
        labels_collection = nx.draw_networkx_labels(G=sub_graph, pos=layout_dict, font_size=10)
    
    # Color each nodes list differently
    else:
        if verbose: display(nodes_list_list)
        color_cycler = nu.get_color_cycler(len(nodes_list_list))
        for nodes_list, fcd in zip(nodes_list_list, color_cycler()):
            if verbose: display(fcd['color'])
            node_color = fcd['color'].reshape(1, -1)
            sub_subgraph = nx.subgraph(sub_graph, nodes_list)
            node_collection = nx.draw_networkx_nodes(G=sub_subgraph, pos=layout_dict, alpha=0.33, node_color=node_color, node_size=150)
            edge_collection = nx.draw_networkx_edges(G=sub_subgraph, pos=layout_dict, alpha=0.25)
            labels_collection = nx.draw_networkx_labels(G=sub_subgraph, pos=layout_dict, font_size=10)

In [ ]:

import itertools
import networkx as nx

# Create data structures to tally the nodes, edges, and weights
edge_tuple_dict = {}
for pair in itertools.combinations(columns_list, 2):
    df = json_stats_df[list(pair)].drop_duplicates().sort_values(list(pair))
    # print()
    # print(*pair)
    eval_list = []
    for record_dict in df.dropna(axis='index', how='any').to_dict(orient='records'):
        values_list = list(record_dict.values())
        name_similarities_df = nu.check_for_typos(*[[v] for v in values_list])
        # print(*values_list)
        eval_list.append(1.0 if (values_list[0] in values_list[1]) else name_similarities_df.max_similarity.squeeze())
    similarity_measure = np.min(eval_list)
    edge_tuple_dict[pair] = similarity_measure#if similarity_measure > 0.08: 

# Create the directed graph
dg = nx.DiGraph()
dg.add_nodes_from(columns_list)
dg.add_weighted_edges_from([(k[0], k[1], 10*v) for k, v in edge_tuple_dict.items()])

In [ ]:

import matplotlib.pyplot as plt
show_subgraph(dg, suptitle='Encouter Layout Features', nodes_list_list=None, node_color=np.array([0.4, 0.4, 0.4, 1.0]), verbose=False)

In [ ]:

columns_list = ['configData_scene', 'configData_scenarioData_name', 'configData_scenarioData_description']
display(json_stats_df[columns_list].drop_duplicates().sort_values(columns_list))
display(json_stats_df.groupby(columns_list, dropna=False).size().to_frame().rename(columns={0: 'record_count'}))

In [ ]:

json_stats_df.sample(5).T

In [ ]:

# Get the unique count of each column
for cn in sorted(json_stats_df.columns): print(cn, json_stats_df[cn].nunique())

In [ ]:

sorted(json_stats_df.columns)

In [ ]:

display(json_stats_df.groupby('responder_type').size().to_frame().rename(columns={0: 'record_count'}).sort_values(
    'record_count', ascending=False
).head(5))

In [ ]:

# Show all patient names
old_patient_names_set = set([
    'Adept Shooter Root', 'Adept Victim Root', 'Broken Bob Root', 'Broken Gloria Root', 'Broken Helga Root', 'Civilian 1 Female Root', 'Civilian 1 Root',
    'Civilian 2 Root', 'Intelligence Officer Root', 'Local Civilian with Internal Bleeding Root', 'Local Soldier 1 Root', 'Marine 1 Male Root', 'Marine 2 Male Root',
    'Marine 3 Male Root', 'Marine 4 Male Root', 'Marine with Leg Amputation Root', 'Marine with Narrative Root', 'NPC 1 Root', 'NPC 2 Root', 'NPC 3 Root',
    'NPC 4 Root', 'NPC Root', 'Navy Soldier 1 Male Root', 'Navy Soldier 2 Male Root', 'Navy Soldier 3 Male Root', 'Navy Soldier 4 Female Root',
    'Navy Solider 4 Female Root', 'Open World Civilian 1 Male Root', 'Open World Civilian 2 Female Root', 'Open World Marine 1 Female Root',
    'Open World Marine 1 Male Root', 'Open World Marine 2 Female Root', 'Open World Marine 2 Male Root', 'Open World Marine 3 Male Root',
    'Open World Marine 4 Male Root', 'Patient U Root', 'Patient V Root', 'Patient W Root', 'Patient X Root', 'Simulation Root', 'Tutorial Military Marine Root',
    'US Soldier 1 Root', 'bystander Root', 'electrician Root', 'patient U Root', 'patient V Root', 'patient W Root', 'patient X Root'
])
mask_series = ~csv_stats_df.patient_id.isnull()
all_patient_names = sorted(csv_stats_df[mask_series].patient_id.unique())
old_patient_names_set.symmetric_difference(set(all_patient_names))

In [ ]:

# Show all the responder types in the registry
df = data_frames_dict['first_responder_master_registry_file_stats_df']
mask_series = ~df.responder_type.isnull()
sorted(df[mask_series].responder_type.unique())

In [ ]:

# Show if you have any "file" columns in the logs data frame
sorted([cn for cn in csv_stats_df.columns if 'file' in cn])

In [ ]:

# Try to base the CACI data on the sessions in their spreadsheet
print(logs_df.shape)
file_path = '../data/xlsx/Metrics_Eval_Participant_data_for_BBAI.xlsx'
participant_data_df = read_excel(file_path)
uuid_columns = [f'Sim{i}' for i in range(1, 3)]
uuid_fn = lambda x: str(x)[:-1] if str(x).endwith('_') else x
json_stats_uuids = []
for participant_id in range(2_024_201, 2_024_223+1):
    mask_series = (participant_data_df.ParticipantID == participant_id)
    if mask_series.any():
        participant_df = participant_data_df[mask_series]
        on_columns = set(json_stats_df.columns).intersection(set(participant_df.columns))
        session_uuids_list = participant_df[uuid_columns].applymap(uuid_fn).values.ravel().tolist()
        mask_series = json_stats_df.session_uuid.isin(session_uuids_list)
        if mask_series.any(): json_stats_uuids.extend(json_stats_df[mask_series].session_uuid.tolist())
    else: print(f'You are missing Participant ID #{participant_id} from the Metrics Eval Participant data for BBAI')
print(logs_df.shape)
# nu.store_objects(metrics_evaluation_open_world_json_stats_df=json_stats_df, verbose=True)
# nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=json_stats_df, verbose=True)

In [ ]:

import datetime

# Filter rows with non-null 'Date' values
mask_series = ~json_stats_df.Date.isnull()
filtered_df = json_stats_df[mask_series]

# Get minimum and maximum dates as datetime objects
min_date = filtered_df['Date'].min()
max_date = filtered_df['Date'].max()

# Format the datetime objects for human-readable output
min_date_str = datetime.datetime.strptime(min_date, '%m/%d/%Y').strftime('%B %d, %Y')
max_date_str = datetime.datetime.strptime(max_date, '%m/%d/%Y').strftime('%B %d, %Y')

# Print the results
print(
    f'A total of 22 participants, representing diverse medical roles, completed the ITM scenarios between {min_date_str} and {max_date_str}.'
    ' Each participant engaged with two separate open world environments.'
)

In [ ]:

# Get the open world action types
import re
import os

action_type_regex = re.compile("^([^,\r\n]+),", re.MULTILINE)
file_ending = '.csv'
logs_path = '../data/logs/Metrics Evaluation Open World'
all_actions_set = set()
for sub_directory, directories_list, files_list in os.walk(logs_path):
    for file_name in files_list:
        if file_name.endswith(file_ending):
            file_path = osp.join(sub_directory, file_name)
            with open(file_path, 'r', encoding=nu.encoding_type) as f:
                text = f.read()
                actions_set = set(action_type_regex.findall(text))
                all_actions_set.update(actions_set)

old_actions_list = [
    'BAG_ACCESS', 'BAG_CLOSED', 'INJURY_RECORD', 'INJURY_TREATED', 'PATIENT_DEMOTED', 'PATIENT_ENGAGED', 'PATIENT_RECORD', 'PULSE_TAKEN',
    'S_A_L_T_WALKED', 'S_A_L_T_WALK_IF_CAN', 'S_A_L_T_WAVED', 'S_A_L_T_WAVE_IF_CAN', 'TAG_APPLIED', 'TAG_DISCARDED', 'TAG_SELECTED', 'TELEPORT',
    'TOOL_APPLIED', 'TOOL_DISCARDED', 'TOOL_HOVER', 'TOOL_SELECTED', 'VOICE_CAPTURE', 'VOICE_COMMAND', 'PLAYER_LOCATION', 'PLAYER_GAZE',
    'SESSION_END', 'SESSION_START'
]
new_actions_list = sorted(all_actions_set.difference(set(old_actions_list)))
print(new_actions_list)